# ETL

# Data Model

# Extract
### Loading Data

I used a CTG exam dataset found on Kaggle from The Journal of Maternal-Fetal Medicine. 
The dataset contains 2,126 rows of 22 features extracted from CTG exams which were classified by expert obstetricians to three fetal health states: normal, suspect, pathological.

In [10]:
# Import Packages
import pandas as pd
import numpy as np
import psycopg2
import warnings
warnings.filterwarnings("ignore")

from cred import password

In [2]:
df=pd.read_csv('fetal_health.csv') 
df.T

,0,1,2,3,4,5,6,7,8,9,...,2116,2117,2118,2119,2120,2121,2122,2123,2124,2125
baseline value,120.0,132.000,133.000,134.000,132.000,134.000,134.000,122.0,122.000,122.000,...,140.000,140.000,140.000,140.000,140.000,140.000,140.000,140.000,140.000,142.000
accelerations,0.0,0.006,0.003,0.003,0.007,0.001,0.001,0.0,0.000,0.000,...,0.004,0.000,0.000,0.000,0.000,0.000,0.001,0.001,0.001,0.002
fetal_movement,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002
uterine_contractions,0.0,0.006,0.008,0.008,0.008,0.010,0.013,0.0,0.002,0.003,...,0.004,0.008,0.006,0.007,0.005,0.007,0.007,0.007,0.006,0.008
light_decelerations,0.0,0.003,0.003,0.003,0.000,0.009,0.008,0.0,0.000,0.000,...,0.000,0.000,0.001,0.001,0.001,0.000,0.000,0.000,0.000,0.000
severe_decelerations,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
prolongued_decelerations,0.0,0.000,0.000,0.000,0.000,0.002,0.003,0.0,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
abnormal_short_term_variability,73.0,17.000,16.000,16.000,16.000,26.000,29.000,83.0,84.000,86.000,...,80.000,79.000,79.000,79.000,77.000,79.000,78.000,79.000,78.000,74.000
mean_value_of_short_term_variability,0.5,2.100,2.100,2.400,2.400,5.900,6.300,0.5,0.500,0.300,...,0.200,0.300,0.500,0.600,0.700,0.200,0.400,0.400,0.400,0.400
percentage_of_time_with_abnormal_long_term_variability,43.0,0.000,0.000,0.000,0.000,0.000,0.000,6.0,5.000,6.000,...,36.000,20.000,26.000,27.000,17.000,25.000,22.000,20.000,27.000,36.000


# Transform
### Cleaning Data

The dataset didn't contain any missing or null values. The only data cleaning we performed is renaming the columns, so it is more readable. I renamed 'baseline value', 'abnormal_short_term_variability, 'mean_value_of_short_term_variability', 'percentage_of_time_with_abnormal_long_term_variability', and 'mean_value_of_long_term_variability' to 'baseline_FHR', 'abnormal_STV', 'mean_STV', 'percentage_duration_LTV', and 'mean_LTV'

In [3]:
#check null values
df.isna().sum()

baseline value                                            0
accelerations                                             0
fetal_movement                                            0
uterine_contractions                                      0
light_decelerations                                       0
severe_decelerations                                      0
prolongued_decelerations                                  0
abnormal_short_term_variability                           0
mean_value_of_short_term_variability                      0
percentage_of_time_with_abnormal_long_term_variability    0
mean_value_of_long_term_variability                       0
histogram_width                                           0
histogram_min                                             0
histogram_max                                             0
histogram_number_of_peaks                                 0
histogram_number_of_zeroes                                0
histogram_mode                          

In [4]:
#rename columns
df.rename(columns={'baseline value':'baseline_FHR',
                   'abnormal_short_term_variability':'abnormal_STV',
                   'mean_value_of_short_term_variability':'mean_STV',
                   'percentage_of_time_with_abnormal_long_term_variability':'percentage_duration_LTV',
                   'mean_value_of_long_term_variability':'mean_LTV'}, inplace=True)

In [5]:
df.columns

Index(['baseline_FHR', 'accelerations', 'fetal_movement',
       'uterine_contractions', 'light_decelerations', 'severe_decelerations',
       'prolongued_decelerations', 'abnormal_STV', 'mean_STV',
       'percentage_duration_LTV', 'mean_LTV', 'histogram_width',
       'histogram_min', 'histogram_max', 'histogram_number_of_peaks',
       'histogram_number_of_zeroes', 'histogram_mode', 'histogram_mean',
       'histogram_median', 'histogram_variance', 'histogram_tendency',
       'fetal_health'],
      dtype='object')

After cleaning the data, I created datafram for deceleration, short term variability, and long term variability, histogram, and fetus_health. Lastly, I exported each DataFrame as .csv files.

### DataFrames

#### Deceleration

In [6]:
deceleration = df[['light_decelerations','severe_decelerations','prolongued_decelerations']]
deceleration["deceleration_ID"] = deceleration.index
deceleration.head()

,light_decelerations,severe_decelerations,prolongued_decelerations,deceleration_ID
0,0.000,0.0,0.0,0
1,0.003,0.0,0.0,1
2,0.003,0.0,0.0,2
3,0.003,0.0,0.0,3
4,0.000,0.0,0.0,4


#### Short Term Variability 

In [7]:
short_term_variability=df[['abnormal_STV','mean_STV']]
short_term_variability["short_term_variability_ID"]=short_term_variability.index
short_term_variability.head()

,abnormal_STV,mean_STV,short_term_variability_ID
0,73.0,0.5,0
1,17.0,2.1,1
2,16.0,2.1,2
3,16.0,2.4,3
4,16.0,2.4,4


#### Long Term Variability

In [8]:
long_term_variability=df[['percentage_duration_LTV','mean_LTV']]
long_term_variability["long_term_variability_ID"]=long_term_variability.index
long_term_variability.head()

,percentage_duration_LTV,mean_LTV,long_term_variability_ID
0,43.0,2.4,0
1,0.0,10.4,1
2,0.0,13.4,2
3,0.0,23.0,3
4,0.0,19.9,4


#### Histogram

In [9]:
histogram = df[['histogram_width',
                   'histogram_min', 'histogram_max', 'histogram_number_of_peaks',
                   'histogram_number_of_zeroes', 'histogram_mode', 'histogram_mean',
                   'histogram_median', 'histogram_variance', 'histogram_tendency']]
histogram["histogram_ID"]=histogram.index
histogram.head()

,histogram_width,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,histogram_ID
0,64.0,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,0
1,130.0,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1
2,130.0,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,2
3,117.0,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,3
4,117.0,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,4


####  Fetus Health
Heart activity of a fetus, it’s movements and the mother’s uterine contractions simultaneously.

In [10]:
fetus_health = df[['baseline_FHR', 'accelerations', 'fetal_movement',
       'uterine_contractions','fetal_health']]
fetus_health["fetus_health_ID"] = fetus_health.index
fetus_health.head()

,baseline_FHR,accelerations,fetal_movement,uterine_contractions,fetal_health,fetus_health_ID
0,120.0,0.000,0.0,0.000,2.0,0
1,132.0,0.006,0.0,0.006,1.0,1
2,133.0,0.003,0.0,0.008,1.0,2
3,134.0,0.003,0.0,0.008,1.0,3
4,132.0,0.007,0.0,0.008,1.0,4


In [11]:
fetus_health["histogram_ID"] = histogram["histogram_ID"]
fetus_health["long_term_variability_ID"] = long_term_variability["long_term_variability_ID"]
fetus_health["short_term_variability_ID"] = short_term_variability["short_term_variability_ID"]
fetus_health["deceleration_ID"] = deceleration["deceleration_ID"]
fetus_health.head()

,baseline_FHR,accelerations,fetal_movement,uterine_contractions,fetal_health,fetus_health_ID,histogram_ID,long_term_variability_ID,short_term_variability_ID,deceleration_ID
0,120.0,0.000,0.0,0.000,2.0,0,0,0,0,0
1,132.0,0.006,0.0,0.006,1.0,1,1,1,1,1
2,133.0,0.003,0.0,0.008,1.0,2,2,2,2,2
3,134.0,0.003,0.0,0.008,1.0,3,3,3,3,3
4,132.0,0.007,0.0,0.008,1.0,4,4,4,4,4


### Export DataFrames

In [12]:
deceleration.to_csv('deceleration.csv',index=False)
short_term_variability.to_csv('short_term_variability.csv',index=False)
long_term_variability.to_csv('long_term_variability.csv',index=False)
histogram.to_csv('histogram.csv',index=False)
fetus_health.to_csv('fetus_health.csv',index=False)

# Load

I loaded all the data into the default Postgres Database. I created tables:  deceleration, short term variability, and long term variability, histogram, and fetus_health.

#### Fetus_Health table

In [13]:
fetus_health.head()

,baseline_FHR,accelerations,fetal_movement,uterine_contractions,fetal_health,fetus_health_ID,histogram_ID,long_term_variability_ID,short_term_variability_ID,deceleration_ID
0,120.0,0.000,0.0,0.000,2.0,0,0,0,0,0
1,132.0,0.006,0.0,0.006,1.0,1,1,1,1,1
2,133.0,0.003,0.0,0.008,1.0,2,2,2,2,2
3,134.0,0.003,0.0,0.008,1.0,3,3,3,3,3
4,132.0,0.007,0.0,0.008,1.0,4,4,4,4,4


In [14]:
conn = psycopg2.connect(host='localhost', dbname='postgres', user='postgres', password=password)
conn.autocommit = True

In [15]:
cur = conn.cursor()
Fetus_Health_table = """Create table Fetus_Health(
                    baseline_FHR          numeric,
                    accelerations         real,
                    fetal_movement        numeric,
                    uterine_contractions  real,
                    fetal_health          numeric,
                    fetus_health_ID       integer  primary key,
                    histogram_ID               integer,
                    long_term_variability_ID   integer,
                    short_term_variability_ID  integer,
                    deceleration_ID            integer
                )"""
cur.execute('DROP TABLE IF EXISTS Fetus_Health')
cur.execute(Fetus_Health_table)
conn.commit()

In [16]:
with open('fetus_health.csv', 'r') as f:
    next(f) # Skip the header row.
    cur.copy_from(f, 'fetus_health', sep=',')
conn.commit()
conn.close()

#### Histogram Table

In [17]:
histogram.head(9)

,histogram_width,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,histogram_ID
0,64.0,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,0
1,130.0,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1
2,130.0,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,2
3,117.0,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,3
4,117.0,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,4
5,150.0,50.0,200.0,5.0,3.0,76.0,107.0,107.0,170.0,0.0,5
6,150.0,50.0,200.0,6.0,3.0,71.0,107.0,106.0,215.0,0.0,6
7,68.0,62.0,130.0,0.0,0.0,122.0,122.0,123.0,3.0,1.0,7
8,68.0,62.0,130.0,0.0,0.0,122.0,122.0,123.0,3.0,1.0,8


In [18]:
conn = psycopg2.connect(host='localhost', dbname='postgres', user='postgres', password=password)
conn.autocommit = True

In [19]:
cur = conn.cursor()
Histogram_table = """Create table Histogram(
                    
                    histogram_width            numeric,
                    histogram_min              numeric,
                    histogram_max              numeric,
                    histogram_number_of_peaks  numeric,
                    histogram_number_of_zeroes numeric,
                    histogram_mode             numeric,
                    histogram_mean             numeric,
                    histogram_median           numeric,
                    histogram_variance         numeric,
                    histogram_tendency         numeric,
                    histogram_ID               integer primary key
                )"""
cur.execute('DROP TABLE IF EXISTS Histogram')
cur.execute(Histogram_table)
conn.commit()

In [20]:
with open('histogram.csv', 'r') as f:
    next(f) # Skip the header row.
    cur.copy_from(f, 'histogram', sep=',')
conn.commit()
conn.close()

#### Deceleration Table

In [21]:
deceleration.head()

,light_decelerations,severe_decelerations,prolongued_decelerations,deceleration_ID
0,0.000,0.0,0.0,0
1,0.003,0.0,0.0,1
2,0.003,0.0,0.0,2
3,0.003,0.0,0.0,3
4,0.000,0.0,0.0,4


In [22]:
conn = psycopg2.connect(host='localhost', dbname='postgres', user='postgres', password=password)
conn.autocommit = True

In [23]:
cur = conn.cursor()
Deceleration_table = """Create table Deceleration(
                    light_decelerations        real,
                    severe_decelerations       numeric,
                    prolongued_decelerations   numeric,
                    deceleration_ID            integer primary key
                )"""
cur.execute('DROP TABLE IF EXISTS Deceleration')
cur.execute(Deceleration_table)
conn.commit()

In [24]:
with open('deceleration.csv', 'r') as f:
    next(f) # Skip the header row.
    cur.copy_from(f, 'deceleration', sep=',')
conn.commit()
conn.close()

#### Short_Term_Variability Table

In [25]:
short_term_variability.head()

,abnormal_STV,mean_STV,short_term_variability_ID
0,73.0,0.5,0
1,17.0,2.1,1
2,16.0,2.1,2
3,16.0,2.4,3
4,16.0,2.4,4


In [26]:
conn = psycopg2.connect(host='localhost', dbname='postgres', user='postgres', password=password)
conn.autocommit = True

In [27]:
cur = conn.cursor()
short_term_variability_table = """Create table short_term_variability(
                    
                    abnormal_STV        numeric,
                    mean_STV            numeric,
                    short_term_variability_ID  integer primary key

                )"""
cur.execute('DROP TABLE IF EXISTS short_term_variability')
cur.execute(short_term_variability_table)
conn.commit()

In [28]:
with open('short_term_variability.csv', 'r') as f:
    next(f) # Skip the header row.
    cur.copy_from(f, 'short_term_variability', sep=',')
conn.commit()
conn.close()

#### Long_Term_Variability Table

In [29]:
long_term_variability.head()

,percentage_duration_LTV,mean_LTV,long_term_variability_ID
0,43.0,2.4,0
1,0.0,10.4,1
2,0.0,13.4,2
3,0.0,23.0,3
4,0.0,19.9,4


In [30]:
conn = psycopg2.connect(host='localhost', dbname='postgres', user='postgres', password=password)
conn.autocommit = True

In [31]:
cur = conn.cursor()
long_term_variability_table = """Create table long_term_variability(
                    percentage_duration_LTV   numeric,
                    mean_LTV                  numeric,
                    long_term_variability_ID  integer primary key

                )"""
cur.execute('DROP TABLE IF EXISTS long_term_variability')
cur.execute(long_term_variability_table)
conn.commit()

In [32]:
with open('long_term_variability.csv', 'r') as f:
    next(f) # Skip the header row.
    cur.copy_from(f, 'long_term_variability', sep=',')
conn.commit()
conn.close()

# Queries

#### Target class count

In [2]:
with psycopg2.connect(host='localhost', dbname='postgres', user='postgres', password=password) as conn:

    query1 = """select fetal_health,count(fetal_health) as target_class_count from fetus_health group by fetal_health """
    
    target_class_count = pd.read_sql_query(query1,conn)

target_class_count.head(5)

,fetal_health,target_class_count
0,1.0,1655
1,2.0,295
2,3.0,176


#### Range of percentage duration LTV

In [5]:
with psycopg2.connect(host='localhost', dbname='postgres', user='postgres', password=password) as conn:

    query2 = """select fetal_health,max(percentage_duration_ltv)-min(percentage_duration_ltv) as range from long_term_variability inner join fetus_health on 
long_term_variability.long_term_variability_id = fetus_health.long_term_variability_id
group by fetal_health """
    
    range_per_ltv = pd.read_sql_query(query2,conn)

range_per_ltv.head(5)

,fetal_health,range
0,1.0,73.0
1,2.0,68.0
2,3.0,91.0


#### Average abnormal STV

In [8]:
with psycopg2.connect(host='localhost', dbname='postgres', user='postgres', password=password) as conn:

    query3 = """ select fetal_health,avg(abnormal_stv) from short_term_variability inner join fetus_health on 
short_term_variability.short_term_variability_id = fetus_health.short_term_variability_id
group by fetal_health"""
    
    avg_abnorm_stv = pd.read_sql_query(query3,conn)

avg_abnorm_stv.head(5)

,fetal_health,avg
0,1.0,42.465861
1,2.0,61.901695
2,3.0,64.539773
